# The Malleable Glyph Tutorial

Malleable Glyph and this library and this tutorial is a living thing. 
If you have suggestions, questions, problems, **please, [e-mail us](https://www.fit.vut.cz/person/herout/)**.

# Installation and Setup

This code installs the mglyph library and some others, used in this tutorial:

In [ ]:
%pip install mglyph
%pip install ipywidgets
%pip install numpy
%pip install scipy

## Connect to Google Drive for storing the Glyphs *(optional)*

When using in colab, this cell connects to the Google Drive (some prompts will appear, make sure to select the right account).  When using outside of colab, the folder is just `MGlyphs`.

Then, it **makes a folder named `MGlyphs`** in `My Drive/MGlyphs`.  That's where generated glyphs will go to make it really easy to collect them on your computer.

Someone may want to set the `outpath` variable to something different?

In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir -p /content/drive/MyDrive/MGlyphs
    outpath = '/content/drive/MyDrive/MGlyphs'
else:
    !mkdir MGlyphs
    outpath = 'MGlyphs'

# Now the Malleable Glyph Tutorial, really

## Basics of making a Glyph

The glyph is essentially a function that takes two arguments:
* `x: float` – the glyph parameter, a number between 0.0 and 100.0
* `canvas: mg.Canvas` – the canvas object, where the glyph is to be drawn

For example:
```
def empty_glyph(x: float, canvas: mg.Canvas) -> None:
    pass
```
## Showing the glyph

`mg.show(my_glyph)` shows a few versions of the glyph at different parameter values (by default 10, 30, 50, 70, 90).

Optional arguments of mg.show() are:
* `x=[[10, 50, 90], [5, 6]]` - specifies which glyph sizes to show. A number like `x=50` asks for showing one size. A list of numbers `x=[20, 50, 80]` shows a row of glyphs with the sizes. An aray of lists shows multiple rows with specifies sizes.
* `shadow=True` - show the shadow so that one can see where the glyph itself ends.  `True` by default.
* `background='whitesmoke'` - background of the compound image with multiple glyphs.
* `scale=2.5` - scales the rendered size.  Higher scales (such as 10) are good for examining details for making samples into slides and publications.

## Drawing a line: `canvas.line()`

`canvas.line((x1, y1), (x2, y2))` draws a line. There are a few optional arguments for customizing the line:
* `width='15p'` – sets the line width.  `'15p'` stands for 15 per mille, size relative to the total glyph size.
* `linecap='round'` – sets the shape of the line ends. Possible line ends: `'butt'`, `'round'`, `'square'`
* `color='blue'` – sets the color. The colors can be for example: [`'darkslateblue'`](https://developer.mozilla.org/en-US/docs/Web/CSS/named-color), `'#483d8b'` is the same color as a hex code, `(0.282, 0.239, 0.545)` is the same color as a triplet of RGB values, and `(0.282, 0.239, 0.545, 0.4)` has 40% alpha (the color is semi-transparent).

## Now the very simplest glyph: `simple_horizontal_line`

In [ ]:
import mglyph as mg

def simple_horizontal_line(x: float, canvas: mg.Canvas) -> None:
    canvas.line((mg.lerp(x, canvas.xcenter, canvas.xleft), canvas.ycenter),    # line start
                (mg.lerp(x, canvas.xcenter, canvas.xright), canvas.ycenter),   # line end
                width='35p', linecap='round', color='darkslateblue')

mg.show(simple_horizontal_line, 
        x=[[5, 25, 50, 75, 95],
           [5, 7, 9, 11, 13]])

## Exporting Glyph for Evaluation: `mg.export()`

* `name=` – title or short description of the glyph. Something like a chapter title in a book.
* `short_name=` – short name for listings and leaderboards of glyphs. Something like a file name.
* `path=` – file to store the glyph into (possibly including the directory path to the file).  The glyph is essentially a `.zip` file with many files in it.  Naming it `xxx.zip` makes sense, naming it `xxx.mglyph` also makes sense.
* `author=`, `email=` – info about the author.  Optional.  Makes much sense for challenge submissions, so that the challenge organizers can contact the author.
* `author_public=False` – Show author's name in the leaderboards and glyph lists.  `False` by default.  The e-mail is never shown in public lists and pages by the challenge organizers; it is there for the purpose of contacting the authors.
* `xvalues=[x / 1000 * 100 for x in range(1001)]` – specification of what *x*-values to export.  By default, 1000 different *x*-values are exported: 0.0, 0.1, 0.2, 0.3, ... 100.0.  You may export less files or more, e.g. `xvalues=[2.0*x for x in range(51)]` exports values: 0, 2, 4, ... 50.

<span style="background-color:Gold">TODO: use np.linspace instead</span>

In [ ]:
import os
mg.export(simple_horizontal_line, xvalues=[2.0*x for x in range(51)],
          name="Simple Horizontal Line", short_name='line',
          path=os.path.join(outpath, "Simple Horizontal Line.mglyph"),
          author="Jane Designoff", email="j.designoff@gmail.com", version="1.0.0")

## Interactive visualization

`mg.interact(my_glyph)` runs an interactive playground, where you can tweak the parameter `x` and see what your glyph looks like.

You can use `mg.interact()` with any glyph to play around.

In [ ]:
mg.interact(simple_horizontal_line)

## Simple quare of lines `simple_scaled_square` and filled 'simple_filled_square'

`mg.lerp(x, from, to)` – interpolates based on the parameter.  When `x==0.0`, the value is equal to `from`, when `x==100.0`, the value is equal to `to`, in between, the value is linearly interpolated.

The canvas knows important values that are usable for interpolating:
* `canvas.xleft`, `canvas.xcenter`, `canvas.xright`
* `canvas.ytop`, `canvas.ycenter`, `canvas.ybottom`
* `canvas.xsize`, `canvas.ysize`
* `canvas.center` – is a tuple of `(canvas.xcenter, canvas.ycenter)`

But for square glyphs, the canvas is set so that the horizontal and vertical coordinates are from -1 to 1.

`canvas.rect(top_left, bottom_right, color='darksalmon')` – draws a rectangle
* `tl: tuple[float, float]` - top-left point
* `br: tuple[float, float]` - bottom-right point
* `color: list[int] | tuple[int] | list[float] | tuple[float] | str = 'black'` - color of lines or fill (depending on `style=`)
* `width: float | str='20p'` - stroke width (`'1p'` is 1/1000 of the glyph dimension)
* `style: str='fill'` - drawing style (`fill` or `stroke`)
* `cap: str='butt'` - style of the line ends  <span style="background-color:Gold">what are other possibilities?</style>
* `join: str='miter'` - style of how the lines are joining <span style="background-color:Gold">what are other possibilities?</style>

In [ ]:
def simple_scaled_square(x: float, canvas: mg.Canvas) -> None:
    tl = (mg.lerp(x, 0.0, -1), mg.lerp(x, 0.0, -1.0))
    br = (mg.lerp(x, 0, 1), mg.lerp(x, 0, 1))
    canvas.rect(tl, br, color='darkslategray', style='stroke', width='27p')

def simple_filled_square(x: float, canvas: mg.Canvas) -> None:
    tl = (mg.lerp(x, 0.0, -1), mg.lerp(x, 0.0, -1.0))
    br = (mg.lerp(x, 0, 1), mg.lerp(x, 0, 1))
    canvas.rect(tl, br, color='darksalmon', style='fill')

mg.show(simple_scaled_square)
mg.show(simple_filled_square)